In [2]:
field_size = 5 # 5x5, always square field
user_record = [] # list of lists for tracking the turns
win_line = [] # list of lists, win conditions that are still possible

In [7]:
# display function
first_row = '┌' + (field_size - 2) * '  -  ' + '┐'
middle_row = '|' + field_size * '   |' 
# last_row = 
        
print(first_row)
print(middle_row)

┌  -    -    -  ┐
|   |   |   |   |   |


In [ ]:
"""
┌ - - - - - ┐
|   |   |   |
|-----------|
|   |   |   |
|-----------|
|   |   |   |
└ - - - - - ┘
"""